In [ ]:
https://github.com/ljh415/AIFFEL/blob/master/Exploration/23.sbr_movielens/SBR_Movielens.ipynb

# 진행순서

### Step 0. 데이터 불러오기
데이터를 불러옵니다.

### Step 1. 데이터의 전처리
위와 같이 간단히 구성해 본 데이터셋을 꼼꼼이 살펴보면서 항목별 기본분석, session length, session time, cleaning 등의 작업을 진행합니다.
특히, 이 데이터셋에서는 Session이 아닌 UserID 단위로 데이터가 생성되어 있으므로, 이를 Session 단위로 어떻게 해석할지에 주의합니다.

### Step 2. 미니 배치의 구성
실습코드 내역을 참고하여 데이터셋과 미니 배치를 구성해 봅시다. Session-Parallel Mini-Batch의 개념에 따라, 학습 속도의 저하가 최소화될 수 있도록 구성합니다.
단, 위 Step 1에서 Session 단위를 어떻게 정의했느냐에 따라서 Session-Parallel Mini-Batch이 굳이 필요하지 않을 수도 있습니다.

### Step 3. 모델 구성
이 부분도 실습코드 내역을 참고하여 다양하게 모델 구조를 시도해볼 수 있습니다.

### Step 4. 모델 학습
다양한 하이퍼파라미터를 변경해 보며 검증해 보도록 합니다. 실습코드에 언급되었던 Recall, MRR 등의 개념들도 함께 관리될 수 있도록 합니다.

### Step 5. 모델 테스트
미리 구성한 테스트셋을 바탕으로 Recall, MRR 을 확인해 봅니다.

# 참고사항
- 여기서 이전 실습내역과 가장 크게 다른 부분은 바로 SessionID 대신 UserID 항목이 들어갔다는 점입니다. 이 데이터셋은 명확한 1회 세션의 SessionID를 포함하지 않고 있습니다. 그래서 이번에는 UserID가 SessionID 역할을 해야 합니다.
- Rating 정보가 포함되어 있습니다. 이전 실습내역에서는 이런 항목이 포함되어 있지 않았으므로, 무시하고 제외할 수 있습니다. 하지만, 직전에 봤던 영화가 맘에 들었는지 여부가 비슷한 영화를 더 고르게 하는 것과 상관이 있을 수도 있습니다. 아울러, Rating이 낮은 데이터를 어떻게 처리할지도 고민해야 합니다.
- Time 항목에는 UTC time 가 포함되어, 1970년 1월 1일부터 경과된 초단위 시간이 기재되어 있습니다.

In [ ]:
# 평가

평가문항	상세기준
1. Movielens 데이터셋을 session based recommendation 관점으로 전처리하는 과정이 체계적으로 진행되었다.
데이터셋의 면밀한 분석을 토대로 세션단위 정의 과정(길이분석, 시간분석)을 합리적으로 수행한 과정이 기술되었다.

2. RNN 기반의 예측 모델이 정상적으로 구성되어 안정적으로 훈련이 진행되었다.
적절한 epoch만큼의 학습이 진행되는 과정에서 train loss가 안정적으로 감소하고, validation 단계에서의 Recall, MRR이 개선되는 것이 확인된다.

3. 세션정의, 모델구조, 하이퍼파라미터 등을 변경해서 실험하여 Recall, MRR 등의 변화추이를 관찰하였다.
3가지 이상의 변화를 시도하고 그 실험결과를 체계적으로 분석하였다.

In [ ]:
# 후기

뒤에 가서는 그냥 노드 베끼기 수준...
추천 어렵다

In [1]:
# 패키지 임포트
import datetime as dt
from pathlib import Path
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 모델링을 위한 패키지 import
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GRU
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

# 데이터 불러오기

In [47]:
# 데이터 불러오기

data_path = Path(os.getenv('HOME')+'/aiffel/yoochoose-data/ml-1m') 
train_path = data_path / 'ratings.dat'

def load_data(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3], dtype={0: np.int32, 1: np.int32, 2: np.int32}, nrows=nrows)
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    return data

data = load_data(train_path, None)
data.sort_values(['UserId', 'Time'], inplace=True)  # data를 id와 시간 순서로 정렬해줍니다.
data

,UserId,ItemId,Rating,Time
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
24,1,2340,3,978300103
...,...,...,...,...
1000019,6040,2917,4,997454429
999988,6040,1921,4,997454464
1000172,6040,1784,3,997454464
1000167,6040,161,3,997454486


In [48]:
# 유저별, 아이템별 데이터 확인
data['UserId'].nunique(), data['ItemId'].nunique()

# 유저는 6040명
# 아이템은 3706개

(6040, 3706)

# 유저별 확인

In [49]:
user = data.groupby('UserId').size()
user

# 유저별 평가가 많은 것은 이상치가 아님
# 그대로 내버려 두자.

UserId
1        53
2       129
3        51
4        21
5       198
       ... 
6036    888
6037    202
6038     20
6039    123
6040    341
Length: 6040, dtype: int64

# rating별 확인

In [50]:
# rating 확인
ratings = data.groupby('Rating').size()
ratings

Rating
1     56174
2    107557
3    261197
4    348971
5    226310
dtype: int64

In [51]:
# 평가 중앙값과 평균을 알아봅시다

# len(ratings)

def rating_mean(x):
    count = 0
    mul = 1
    sum_all = 0
    for z in range(1,len(x)+1):
        count += x[z]
        sum_all += mul * x[z]
        mul += 1
        
    print("총 데이터 개수 :", count)
    print("총 별점 합 :", sum_all)
    return sum_all / count


print("평균 :",rating_mean(ratings))


# 4점 밑으로는 버린다!!

총 데이터 개수 : 1000209
총 별점 합 : 3582313
평균 : 3.581564453029317


In [52]:
# 3점 이하로는 데이터를 삭제해도 될 듯하다. 
# 어차피 왓챠에서도 예상평가 3점 이하면 안본다.
re_data = data[data['Rating'] > 3]
re_data

,UserId,ItemId,Rating,Time
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
36,1,1836,5,978300172
...,...,...,...,...
1000119,6040,3671,4,997454367
999923,6040,232,5,997454398
1000019,6040,2917,4,997454429
999988,6040,1921,4,997454464


# 영화별 확인

In [53]:
# 영화별 평가 수
from collections import Counter

movie = Counter(re_data['ItemId'])
movie_items = movie.items()
movie_items = sorted(movie_items, key=lambda x: x[1], reverse=True)
movie_items

# 적게 등장한 영화를 보고 짤라야 하나?
# 보통 적게 본 영화는 사람들이 좋아하지 않는 독립영화일수도.

[(2858, 2853),
 (260, 2622),
 (1196, 2510),
 (2028, 2260),
 (1198, 2260),
 (593, 2252),
 (2571, 2171),
 (2762, 2163),
 (1210, 2127),
 (608, 2074),
 (527, 2071),
 (318, 2046),
 (589, 2044),
 (858, 1989),
 (110, 1977),
 (1197, 1924),
 (1270, 1910),
 (2396, 1877),
 (1617, 1876),
 (296, 1770),
 (2997, 1759),
 (480, 1730),
 (1240, 1683),
 (356, 1668),
 (1265, 1661),
 (1, 1655),
 (1580, 1644),
 (1097, 1643),
 (1214, 1623),
 (457, 1615),
 (50, 1608),
 (2716, 1548),
 (1193, 1519),
 (3578, 1508),
 (541, 1485),
 (1221, 1444),
 (912, 1434),
 (1200, 1421),
 (1259, 1420),
 (919, 1385),
 (1213, 1370),
 (1136, 1364),
 (1036, 1332),
 (3114, 1302),
 (1610, 1297),
 (1291, 1297),
 (2791, 1284),
 (924, 1270),
 (1387, 1266),
 (1704, 1256),
 (2916, 1248),
 (34, 1234),
 (1307, 1211),
 (750, 1207),
 (2355, 1192),
 (1304, 1170),
 (908, 1170),
 (2918, 1168),
 (2000, 1168),
 (1225, 1155),
 (3175, 1145),
 (2628, 1132),
 (2599, 1130),
 (2987, 1111),
 (2804, 1105),
 (2959, 1096),
 (32, 1083),
 (1968, 1067),
 (1394,

In [54]:
# 가장 적게 본 영화 목록

sort_movie = sorted(movie_items, key=lambda x : x[1])
sort_movie

# 50개 이하면 짜를까?
# 고민해보자

[(3280, 1),
 (1522, 1),
 (774, 1),
 (1891, 1),
 (717, 1),
 (2685, 1),
 (3187, 1),
 (228, 1),
 (2754, 1),
 (3647, 1),
 (734, 1),
 (2251, 1),
 (2244, 1),
 (792, 1),
 (3800, 1),
 (3346, 1),
 (220, 1),
 (1102, 1),
 (3482, 1),
 (758, 1),
 (3542, 1),
 (712, 1),
 (2869, 1),
 (3205, 1),
 (1861, 1),
 (401, 1),
 (3131, 1),
 (3294, 1),
 (2242, 1),
 (1877, 1),
 (2765, 1),
 (2449, 1),
 (3085, 1),
 (2487, 1),
 (3944, 1),
 (687, 1),
 (212, 1),
 (1335, 1),
 (3573, 1),
 (2855, 1),
 (3313, 1),
 (1773, 1),
 (2464, 1),
 (3277, 1),
 (3295, 1),
 (1652, 1),
 (706, 1),
 (696, 1),
 (398, 1),
 (989, 1),
 (192, 1),
 (2679, 1),
 (3058, 1),
 (33, 1),
 (3887, 1),
 (139, 1),
 (1878, 1),
 (1915, 1),
 (2887, 1),
 (815, 1),
 (789, 1),
 (3322, 1),
 (1830, 1),
 (3881, 1),
 (2777, 1),
 (1325, 1),
 (130, 1),
 (1787, 1),
 (1890, 1),
 (1886, 1),
 (2368, 1),
 (1495, 1),
 (1826, 1),
 (470, 1),
 (1490, 1),
 (1470, 1),
 (981, 1),
 (2955, 1),
 (679, 1),
 (3777, 1),
 (3381, 1),
 (1555, 1),
 (3818, 1),
 (623, 1),
 (2223, 1),
 (3172

In [55]:
# sort_movie[2][0]

# 지워야 할 영화 리스트를 뽑아서 몇 개인지 확인해 봅시다.
delete_list = []
for x in range(len(sort_movie)) :
    value = sort_movie[x][1]
    if value < 50 :
        delete_list.append(sort_movie[x][0])
        
print("지워야 할 영화 리스트 개수 :",len(delete_list))

지워야 할 영화 리스트 개수 : 1770


In [56]:
# 전체 데이터에서 해당 리스트에 해당하는 데이터를 지운 뒤 데이터 개수를 파악해 봅시다.

# # 오류 발생 
# for x in range(len(re_data)):
#     if re_data.iloc[x][1] in delete_list :
#         re_data.drop(x)
                
# head(re_data)


# 일단 데이터도 적으니까 못 먹어도 고!

## 여기서부터는 시간 정보 확인

In [57]:
# 우선 현재 데이터 개수 확인
re_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 575281 entries, 31 to 1000042
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   UserId  575281 non-null  int32
 1   ItemId  575281 non-null  int32
 2   Rating  575281 non-null  int32
 3   Time    575281 non-null  int64
dtypes: int32(3), int64(1)
memory usage: 15.4 MB


In [59]:
# 사칙연산이 가능한 datetime으로 바꿔봅시다.

# datetime으로 바꿔봅시다.
from time import localtime, strftime
from datetime import datetime, timezone


re_data['Time'] = re_data['Time'].apply(lambda x:dt.datetime.utcfromtimestamp(int(x)))
re_data.head()


# new_list = []
# for now in data['Time']:
#     local_tuple = localtime(now)
#     time_format = '%Y-%m-%d %H:%M:%S'
#     time_str = strftime(time_format, local_tuple)
#     new_list.append(time_str)
    
# data['Time'] = new_list
# data['Time'] = pd.to_datetime(data['Time'], format='%Y-%m-%d %H:%M:%S', errors='raise')
# data.head()

,UserId,ItemId,Rating,Time
31,1,3186,4,2000-12-31 22:00:19
22,1,1270,5,2000-12-31 22:00:55
27,1,1721,4,2000-12-31 22:00:55
37,1,1022,5,2000-12-31 22:00:55
36,1,1836,5,2000-12-31 22:02:52


In [60]:
# 첫번째 기록의 시간과 마지막 기록의 시간
oldest, latest = re_data['Time'].min(), re_data['Time'].max()
print(oldest) 
print(latest)

2000-04-25 23:05:32
2003-02-28 17:49:50


In [42]:
month_ago = latest - dt.timedelta(365)     # 최종 날짜로부터 365일 이전 날짜를 구한다.  
data = re_data[re_data['Time'] > month_ago]   # 방금 구한 날짜 이후의 데이터만 모은다. 
data 

# 1만1천개 남짓... 그래도 들어가..?
# 이대로 가면 망할 거 같아. 그냥 타임 데이터는 다 쓰자.
# 날짜는 전체 다 쓰자. (re_data)를 쓴다.
# 대신 최신에 있는 데이터를 test, validation으로 써서 정확도를 높이자.

,UserId,ItemId,Rating,Time
5170,36,1387,5,2002-03-12 03:46:59
5267,36,1201,4,2002-03-12 03:46:59
5122,36,1291,5,2002-03-12 03:47:16
5123,36,2167,5,2002-03-12 03:48:25
5290,36,2951,4,2002-03-12 03:48:25
...,...,...,...,...
992523,5996,719,4,2002-04-29 20:13:03
992579,5996,2605,5,2002-04-29 20:13:03
992724,5996,3707,5,2002-04-29 20:16:15
992553,5996,781,5,2002-04-29 20:18:44


In [62]:
# Time 데이터를 순서대로!!

re_data.sort_values(by=['Time'], inplace=True) # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html
print(re_data.shape)
display(re_data)

(575281, 4)


,UserId,ItemId,Rating,Time
1000138,6040,858,4,2000-04-25 23:05:32
999873,6040,593,5,2000-04-25 23:05:54
1000153,6040,2384,4,2000-04-25 23:05:54
1000007,6040,1961,4,2000-04-25 23:06:17
1000192,6040,2019,5,2000-04-25 23:06:17
...,...,...,...,...
825497,4958,2453,4,2003-02-28 17:44:20
825526,4958,3489,4,2003-02-28 17:45:20
825438,4958,1407,5,2003-02-28 17:47:23
825724,4958,3264,4,2003-02-28 17:49:08


In [65]:
print("전체의 0.7 :", 575281 * 0.7)
print("전체의 0.2 :", 575281 * 0.2)
print("전체의 0.1 :", 575281 * 0.1)

# 대충 4십만, 1십만, 5만이다.

전체의 0.7 : 402696.69999999995
전체의 0.2 : 115056.20000000001
전체의 0.1 : 57528.100000000006


# 데이터 전처리로 들어가 봅시다.

## 여기서부터는 훈련, 유효성, 테스트 데이터를 만든다

In [66]:
# 훈련데이터와 유효성검증 데이터 분리 함수

# 데이터 시간을 기준으로 나눈다. ==> why?
# 추천시스템은 '현재'의 취향을 반영해 전달해야 함
# 때문에 최근 데이터로만 진행해야 한다.
# 1달 전 성능이 좋았던 모델도 지금은 안맞을 수 있음

# 하지만 지금 데이터가 많지 않고, Time 데이터 순으로 되어 있으니, 그냥 비율로 자르자.

# split
data_train = data[:400000]
data_val = data[400000:500000]
data_test = data[500000:]

In [67]:
# SessionId -> UserId로 변경
def stats_info(data: pd.DataFrame, status: str):
    print(f'* {status} Set Stats Info\n'
          f'\t Events: {len(data)}\n'
          f'\t Users (Sessions): {data["UserId"].nunique()}\n'
          f'\t Items: {data["ItemId"].nunique()}\n'
          f'\t First Time : {data["Time"].min()}\n'
          f'\t Last Time : {data["Time"].max()}\n')

In [68]:
stats_info(data_train, 'train')
stats_info(data_val, 'valid')
stats_info(data_test, 'test')

## 뭐지?? test 쪽에 유저가 엄청 맣네.

* train Set Stats Info
	 Events: 400000
	 Users (Sessions): 2380
	 Items: 3602
	 First Time : 974329852
	 Last Time : 1046388675

* valid Set Stats Info
	 Events: 100000
	 Users (Sessions): 690
	 Items: 3249
	 First Time : 969951420
	 Last Time : 1045690373

* test Set Stats Info
	 Events: 500209
	 Users (Sessions): 2972
	 Items: 3601
	 First Time : 956703932
	 Last Time : 1046454590



In [69]:
# train set에 없는 아이템이 val, test기간에 생길 수 있으므로 train data를 기준으로 인덱싱합니다.
id2idx = {item_id : index for index, item_id in enumerate(data_train['ItemId'].unique())}

def indexing(df, id2idx):
    df['item_idx'] = df['ItemId'].map(lambda x: id2idx.get(x, -1))  # id2idx에 없는 아이템은 모르는 값(-1) 처리 해줍니다.
    return df

data_train = indexing(data_train, id2idx)
data_val = indexing(data_val, id2idx)
data_test = indexing(data_test, id2idx)

In [70]:
save_path = data_path / 'processed'
save_path.mkdir(parents=True, exist_ok=True)

data_train.to_pickle(save_path / 'train.pkl')
data_val.to_pickle(save_path / 'valid.pkl')
data_test.to_pickle(save_path / 'test.pkl')

# 이제 모델 훈련 들어가봅시다.

# 먼저 모델 구성

In [71]:
# SessionId -> UserId로 변경
class SessionDataset:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, data):
        self.df = data
        self.click_offsets = self.get_click_offsets()
        self.session_idx = np.arange(self.df['UserId'].nunique())  # indexing to SessionId

    def get_click_offsets(self):
        """
        Return the indexes of the first click of each session IDs,
        """
        offsets = np.zeros(self.df['UserId'].nunique() + 1, dtype=np.int32)
        offsets[1:] = self.df.groupby('UserId').size().cumsum()
        return offsets

In [72]:
tr_dataset = SessionDataset(data_train)
tr_dataset.df.head(10)

,UserId,ItemId,Rating,Time,item_idx
31,1,3186,4,978300019,0
22,1,1270,5,978300055,1
27,1,1721,4,978300055,2
37,1,1022,5,978300055,3
24,1,2340,3,978300103,4
36,1,1836,5,978300172,5
3,1,3408,4,978300275,6
7,1,2804,5,978300719,7
47,1,1207,4,978300719,8
0,1,1193,5,978300760,9


In [73]:
tr_dataset.click_offsets

array([     0,     53,    182, ..., 399642, 399668, 400000], dtype=int32)

In [74]:
tr_dataset.session_idx

array([   0,    1,    2, ..., 2377, 2378, 2379])

In [77]:
class SessionDataLoader:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, dataset: SessionDataset, batch_size=50):
        self.dataset = dataset
        self.batch_size = batch_size

    def __iter__(self):
        """ Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,):  Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the sessions to be terminated
        """

        start, end, mask, last_session, finished = self.initialize()  # initialize 메소드에서 확인해주세요.
        """
        start : Index Where Session Start
        end : Index Where Session End
        mask : indicator for the sessions to be terminated
        """

        while not finished:
            min_len = (end - start).min() - 1  # Shortest Length Among Sessions
            for i in range(min_len):
                # Build inputs & targets
                inp = self.dataset.df['item_idx'].values[start + i]
                target = self.dataset.df['item_idx'].values[start + i + 1]
                yield inp, target, mask

            start, end, mask, last_session, finished = self.update_status(start, end, min_len, last_session, finished)

    def initialize(self):
        first_iters = np.arange(self.batch_size)    # 첫 배치에 사용할 세션 Index를 가져옵니다.
        last_session = self.batch_size - 1    # 마지막으로 다루고 있는 세션 Index를 저장해둡니다.
        start = self.dataset.click_offsets[self.dataset.session_idx[first_iters]]       # data 상에서 session이 시작된 위치를 가져옵니다.
        end = self.dataset.click_offsets[self.dataset.session_idx[first_iters] + 1]  # session이 끝난 위치 바로 다음 위치를 가져옵니다.
        mask = np.array([])   # session의 모든 아이템을 다 돌은 경우 mask에 추가해줄 것입니다.
        finished = False         # data를 전부 돌았는지 기록하기 위한 변수입니다.
        return start, end, mask, last_session, finished

    def update_status(self, start: np.ndarray, end: np.ndarray, min_len: int, last_session: int, finished: bool):  
        # 다음 배치 데이터를 생성하기 위해 상태를 update합니다.
        
        start += min_len   # __iter__에서 min_len 만큼 for문을 돌았으므로 start를 min_len 만큼 더해줍니다.
        mask = np.arange(self.batch_size)[(end - start) == 1]  
        # end는 다음 세션이 시작되는 위치인데 start와 한 칸 차이난다는 것은 session이 끝났다는 뜻입니다. mask에 기록해줍니다.

        for i, idx in enumerate(mask, start=1):  # mask에 추가된 세션 개수만큼 새로운 세션을 돌것입니다.
            new_session = last_session + i  
            if new_session > self.dataset.session_idx[-1]:  # 만약 새로운 세션이 마지막 세션 index보다 크다면 모든 학습데이터를 돈 것입니다.
                finished = True
                break
            # update the next starting/ending point
            start[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session]]     # 종료된 세션 대신 새로운 세션의 시작점을 기록합니다.
            end[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session] + 1]

        last_session += len(mask)  # 마지막 세션의 위치를 기록해둡니다.
        return start, end, mask, last_session, finished

In [78]:
tr_data_loader = SessionDataLoader(tr_dataset, batch_size=4)
tr_dataset.df.head(15)

,UserId,ItemId,Rating,Time,item_idx
31,1,3186,4,978300019,0
22,1,1270,5,978300055,1
27,1,1721,4,978300055,2
37,1,1022,5,978300055,3
24,1,2340,3,978300103,4
36,1,1836,5,978300172,5
3,1,3408,4,978300275,6
7,1,2804,5,978300719,7
47,1,1207,4,978300719,8
0,1,1193,5,978300760,9


In [79]:
iter_ex = iter(tr_data_loader)

In [80]:
inputs, labels, mask =  next(iter_ex)
print(f'Model Input Item Idx are : {inputs}')
print(f'Label Item Idx are : {"":5} {labels}')
print(f'Previous Masked Input Idx are {mask}')

Model Input Item Idx are : [ 0 53 65 54]
Label Item Idx are :       [ 1 54 62 24]
Previous Masked Input Idx are []


In [81]:
# SessionId -> UserId로 변경
class Args:
    def __init__(self, tr, val, test, batch_size, hsz, drop_rate, lr, epochs, k):
        self.tr = tr
        self.val = val
        self.test = test
        self.num_items = tr['ItemId'].nunique()
        self.num_sessions = tr['UserId'].nunique()
        self.batch_size = batch_size
        self.hsz = hsz
        self.drop_rate = drop_rate
        self.lr = lr
        self.epochs = epochs
        self.k = k

args = Args(data_train, data_val, data_test, batch_size=256, hsz=50, drop_rate=0.1, lr=0.001, epochs=5, k=20)

In [83]:
def create_model(args):
    inputs = Input(batch_shape=(args.batch_size, 1, args.num_items))
    gru, _ = GRU(args.hsz, stateful=True, return_state=True, name='GRU')(inputs)
    dropout = Dropout(args.drop_rate)(gru)
    predictions = Dense(args.num_items, activation='softmax')(dropout)
    model = Model(inputs=inputs, outputs=[predictions])
    model.compile(loss=categorical_crossentropy, optimizer=Adam(args.lr), metrics=['accuracy'])
    model.summary()
    return model

In [84]:
model = create_model(args)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(256, 1, 3602)]          0         
_________________________________________________________________
GRU (GRU)                    [(256, 50), (256, 50)]    548100    
_________________________________________________________________
dropout (Dropout)            (256, 50)                 0         
_________________________________________________________________
dense (Dense)                (256, 3602)               183702    
Total params: 731,802
Trainable params: 731,802
Non-trainable params: 0
_________________________________________________________________


In [87]:
def mrr_k(pred, truth: int, k: int):
    indexing = np.where(pred[:k] == truth)[0]
    if len(indexing) > 0:
        return 1 / (indexing[0] + 1)
    else:
        return 0


def recall_k(pred, truth: int, k: int) -> int:
    answer = truth in pred[:k]
    return int(answer)

In [88]:
# SessionId -> UserId
def train_model(model, args):
    train_dataset = SessionDataset(args.tr)
    train_loader = SessionDataLoader(train_dataset, batch_size=args.batch_size)

    for epoch in range(1, args.epochs + 1):
        total_step = len(args.tr) - args.tr['UserId'].nunique()
        tr_loader = tqdm(train_loader, total=total_step // args.batch_size, desc='Train', mininterval=1)
        for feat, target, mask in tr_loader:
            reset_hidden_states(model, mask)  # 종료된 session은 hidden_state를 초기화합니다. 아래 메서드에서 확인해주세요.

            input_ohe = to_categorical(feat, num_classes=args.num_items)
            input_ohe = np.expand_dims(input_ohe, axis=1)
            target_ohe = to_categorical(target, num_classes=args.num_items)

            result = model.train_on_batch(input_ohe, target_ohe)
            tr_loader.set_postfix(train_loss=result[0], accuracy = result[1])

        val_recall, val_mrr = get_metrics(args.val, model, args, args.k)  # valid set에 대해 검증합니다.

        print(f"\t - Recall@{args.k} epoch {epoch}: {val_recall:3f}")
        print(f"\t - MRR@{args.k}    epoch {epoch}: {val_mrr:3f}\n")


def reset_hidden_states(model, mask):
    gru_layer = model.get_layer(name='GRU')  # model에서 gru layer를 가져옵니다.
    hidden_states = gru_layer.states[0].numpy()  # gru_layer의 parameter를 가져옵니다.
    for elt in mask:  # mask된 인덱스 즉, 종료된 세션의 인덱스를 돌면서
        hidden_states[elt, :] = 0  # parameter를 초기화 합니다.
    gru_layer.reset_states(states=hidden_states)


def get_metrics(data, model, args, k: int):  # valid셋과 test셋을 평가하는 코드입니다. 
                                             # train과 거의 같지만 mrr, recall을 구하는 라인이 있습니다.
    dataset = SessionDataset(data)
    loader = SessionDataLoader(dataset, batch_size=args.batch_size)
    recall_list, mrr_list = [], []

    total_step = len(data) - data['UserId'].nunique()
    for inputs, label, mask in tqdm(loader, total=total_step // args.batch_size, desc='Evaluation', mininterval=1):
        reset_hidden_states(model, mask)
        input_ohe = to_categorical(inputs, num_classes=args.num_items)
        input_ohe = np.expand_dims(input_ohe, axis=1)

        pred = model.predict(input_ohe, batch_size=args.batch_size)
        pred_arg = tf.argsort(pred, direction='DESCENDING')  # softmax 값이 큰 순서대로 sorting 합니다.

        length = len(inputs)
        recall_list.extend([recall_k(pred_arg[i], label[i], k) for i in range(length)])
        mrr_list.extend([mrr_k(pred_arg[i], label[i], k) for i in range(length)])

    recall, mrr = np.mean(recall_list), np.mean(mrr_list)
    return recall, mrr

In [89]:
train_model(model, args)

Train:   0%|          | 0/1553 [00:00<?, ?it/s, accuracy=0.0469, train_loss=6.14] 

	 - Recall@20 epoch 1: 0.204468
	 - MRR@20    epoch 1: 0.051563



Train:   0%|          | 0/1553 [00:00<?, ?it/s, accuracy=0.0625, train_loss=5.8]  

	 - Recall@20 epoch 2: 0.266792
	 - MRR@20    epoch 2: 0.072988



Train:   0%|          | 0/1553 [00:00<?, ?it/s, accuracy=0.0703, train_loss=5.58] 

	 - Recall@20 epoch 3: 0.299562
	 - MRR@20    epoch 3: 0.086390



Train:   0%|          | 0/1553 [00:00<?, ?it/s, accuracy=0.0742, train_loss=5.45] 

	 - Recall@20 epoch 4: 0.319747
	 - MRR@20    epoch 4: 0.095969



Evaluation:  57%|█████▋    | 221/387 [02:17<01:43,  1.61it/s]

	 - Recall@20 epoch 5: 0.334046
	 - MRR@20    epoch 5: 0.102820



In [90]:
def test_model(model, args, test):
    test_recall, test_mrr = get_metrics(test, model, args, 20)
    print(f"\t - Recall@{args.k}: {test_recall:3f}")
    print(f"\t - MRR@{args.k}: {test_mrr:3f}\n")

test_model(model, args, data_test)

Evaluation:  91%|█████████ | 1758/1942 [19:16<02:01,  1.52it/s]

	 - Recall@20: 0.266263
	 - MRR@20: 0.073596

